# 参数初始化

## 使用 NumPy 来初始化

In [6]:
import numpy as np
import torch
from torch import nn

In [13]:
# 定义一个 Sequential 模型
net1 = nn.Sequential(
    nn.Linear(30, 40),
    nn.ReLU(),
    nn.Linear(40, 50),
    nn.ReLU(),
    nn.Linear(50, 10)
)

In [14]:
# 访问第一层的参数
w1 = net1[0].weight
b1 = net1[0].bias

In [15]:
print(w1)

Parameter containing:
-0.0221 -0.0171  0.1221  ...  -0.0452 -0.1715 -0.0637
-0.0922 -0.1111  0.0822  ...   0.0316  0.1020 -0.0585
-0.0830  0.1037 -0.0572  ...  -0.1465 -0.1049 -0.0566
          ...             ⋱             ...          
 0.1485  0.1137  0.1745  ...   0.0073  0.0887  0.1143
 0.1634 -0.1478  0.0930  ...   0.1418 -0.0501  0.1266
 0.0943 -0.1595 -0.1742  ...  -0.1531  0.0786 -0.1594
[torch.FloatTensor of size 40x30]



In [16]:
# 定义一个 Tensor 直接对其进行替换
net1[0].weight.data = torch.from_numpy(np.random.uniform(3, 5, size=(40, 30)))

In [17]:
print(net1[0].weight)

Parameter containing:
 4.6904  3.5478  4.0254  ...   3.6078  4.6897  4.8285
 4.6349  3.4475  4.8485  ...   3.8712  3.9396  4.7797
 3.0177  3.4870  4.5741  ...   4.6718  4.2548  4.6343
          ...             ⋱             ...          
 3.3116  3.2907  4.5550  ...   3.5882  4.4668  3.6532
 4.2998  4.6337  3.8836  ...   3.1220  4.0567  4.3605
 4.3862  4.5433  4.1909  ...   4.2792  4.7513  3.7076
[torch.DoubleTensor of size 40x30]



In [18]:
for layer in net1:
    if isinstance(layer, nn.Linear): # 判断是否是线性层
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 
        # 定义为均值为 0，方差为 0.5 的正态分布

In [28]:
class sim_net(nn.Module):
    def __init__(self):
        super(sim_net, self).__init__()
        self.l1 = nn.Sequential(
            nn.Linear(30, 40),
            nn.ReLU()
        )
        
        self.l1[0].weight.data = torch.randn(30, 40) # 直接对某一层初始化
        
        self.l2 = nn.Sequential(
            nn.Linear(40, 50),
            nn.ReLU()
        )
        
        self.l3 = nn.Sequential(
            nn.Linear(50, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.l1(x)
        x =self.l2(x)
        x = self.l3(x)
        return x

In [29]:
net2 = sim_net()

In [30]:
# 访问 children
for i in net2.children():
    print(i)

Sequential(
  (0): Linear(in_features=30, out_features=40)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=40, out_features=50)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=50, out_features=10)
  (1): ReLU()
)


In [31]:
# 访问 modules
for i in net2.modules():
    print(i)

sim_net(
  (l1): Sequential(
    (0): Linear(in_features=30, out_features=40)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=40, out_features=50)
    (1): ReLU()
  )
  (l3): Sequential(
    (0): Linear(in_features=50, out_features=10)
    (1): ReLU()
  )
)
Sequential(
  (0): Linear(in_features=30, out_features=40)
  (1): ReLU()
)
Linear(in_features=30, out_features=40)
ReLU()
Sequential(
  (0): Linear(in_features=40, out_features=50)
  (1): ReLU()
)
Linear(in_features=40, out_features=50)
ReLU()
Sequential(
  (0): Linear(in_features=50, out_features=10)
  (1): ReLU()
)
Linear(in_features=50, out_features=10)
ReLU()


In [33]:
for layer in net2.modules():
    if isinstance(layer, nn.Linear):
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 

## torch.nn.init

In [34]:
from torch.nn import init

In [38]:
print(net1[0].weight)

Parameter containing:
 0.2051 -0.4551  0.7049  ...   0.5223 -0.7658 -0.1899
 0.2562  0.2797  0.0012  ...  -0.5278 -0.4887 -0.8263
 0.4582 -0.1433  0.5009  ...   0.1000 -0.5663  0.1605
          ...             ⋱             ...          
 0.8715  0.4053  0.3679  ...  -0.4733 -0.6270 -0.3325
-0.1898  0.6608  0.1111  ...   0.2294  0.2603 -0.0200
-0.3035  0.1876 -0.5422  ...   0.0505  0.6244 -0.2368
[torch.DoubleTensor of size 40x30]



In [39]:
init.xavier_uniform(net1[0].weight)

Parameter containing:
-0.0449 -0.2140  0.2820  ...  -0.2266  0.0365 -0.1897
-0.0313  0.1128  0.1789  ...  -0.1731  0.0590 -0.1085
-0.0347 -0.1429 -0.1646  ...   0.0212  0.1731 -0.0251
          ...             ⋱             ...          
 0.0902 -0.1555  0.0562  ...  -0.0109 -0.2192 -0.1540
-0.1491 -0.2610 -0.2453  ...   0.2201  0.2257  0.1047
 0.0297  0.1414 -0.0139  ...  -0.1209 -0.0193 -0.1731
[torch.DoubleTensor of size 40x30]

In [40]:
print(net1[0].weight)

Parameter containing:
-0.0449 -0.2140  0.2820  ...  -0.2266  0.0365 -0.1897
-0.0313  0.1128  0.1789  ...  -0.1731  0.0590 -0.1085
-0.0347 -0.1429 -0.1646  ...   0.0212  0.1731 -0.0251
          ...             ⋱             ...          
 0.0902 -0.1555  0.0562  ...  -0.0109 -0.2192 -0.1540
-0.1491 -0.2610 -0.2453  ...   0.2201  0.2257  0.1047
 0.0297  0.1414 -0.0139  ...  -0.1209 -0.0193 -0.1731
[torch.DoubleTensor of size 40x30]

